In [2]:
import zipfile
import os
import cv2
import numpy as np
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings('ignore')

# Path to the zip file
zip_path = r"C:\Users\91939\Downloads\archive (2).zip"
# Path where the extracted files will be stored
extract_path = r"C:\Users\91939\Downloads\archive (2)"
# dataset link : https://www.kaggle.com/datasets/gti-upm/leapgestrecog
# Ensure the extract_path directory exists
if not os.path.exists(extract_path):
    os.makedirs(extract_path)

# Extract the zip file
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

print("Data extraction completed.")

# Paths
data_path = os.path.join(extract_path, 'leapGestRecog')
gesture_names = ["01_palm", "02_l", "03_fist", "04_fist_moved", "05_thumb", "06_index", "07_ok", "08_palm_moved", "09_c", "10_down"]

# Verify directory structure
if not os.path.exists(data_path):
    print(f"Directory does not exist: {data_path}")
else:
    for i in range(10):  # Folders 00 to 09
        for gesture in gesture_names:
            folder_path = os.path.join(data_path, f'{i:02d}', gesture)
            if not os.path.exists(folder_path):
                print(f"Directory does not exist: {folder_path}")
            else:
                print(f"Directory exists: {folder_path}")

# Parameters
img_size = 32  # Resize images to 32x32 pixels to save memory
num_classes = len(gesture_names)

# Load images and labels
images = []
labels = []

for label, gesture in enumerate(gesture_names):
    for i in range(10):  # Folders 00 to 09
        folder_path = os.path.join(data_path, f'{i:02d}', gesture)
        if os.path.exists(folder_path):
            for img_file in os.listdir(folder_path):
                img_path = os.path.join(folder_path, img_file)
                img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
                img = cv2.resize(img, (img_size, img_size))
                images.append(img)
                labels.append(label)

# Convert to numpy arrays
images = np.array(images, dtype=np.float32).reshape(-1, img_size, img_size, 1)
labels = to_categorical(np.array(labels), num_classes=num_classes)

# Normalize pixel values
images = images / 255.0

# Split into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(images, labels, test_size=0.2, random_state=42)

print(f"Data preprocessing completed. Number of training samples: {len(X_train)}, Number of validation samples: {len(X_val)}")

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(img_size, img_size, 1)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(num_classes, activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()

from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

# Callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
model_checkpoint = ModelCheckpoint('gesture_recognition_model.keras', save_best_only=True)

# Train the model
history = model.fit(X_train, y_train, epochs=30, validation_data=(X_val, y_val), batch_size=32, callbacks=[early_stopping, model_checkpoint])

print("Model training completed.")

# Evaluate the model
val_loss, val_accuracy = model.evaluate(X_val, y_val)
print(f"Validation Loss: {val_loss}, Validation Accuracy: {val_accuracy}")


Data extraction completed.
Directory exists: C:\Users\91939\Downloads\archive (2)\leapGestRecog\00\01_palm
Directory exists: C:\Users\91939\Downloads\archive (2)\leapGestRecog\00\02_l
Directory exists: C:\Users\91939\Downloads\archive (2)\leapGestRecog\00\03_fist
Directory exists: C:\Users\91939\Downloads\archive (2)\leapGestRecog\00\04_fist_moved
Directory exists: C:\Users\91939\Downloads\archive (2)\leapGestRecog\00\05_thumb
Directory exists: C:\Users\91939\Downloads\archive (2)\leapGestRecog\00\06_index
Directory exists: C:\Users\91939\Downloads\archive (2)\leapGestRecog\00\07_ok
Directory exists: C:\Users\91939\Downloads\archive (2)\leapGestRecog\00\08_palm_moved
Directory exists: C:\Users\91939\Downloads\archive (2)\leapGestRecog\00\09_c
Directory exists: C:\Users\91939\Downloads\archive (2)\leapGestRecog\00\10_down
Directory exists: C:\Users\91939\Downloads\archive (2)\leapGestRecog\01\01_palm
Directory exists: C:\Users\91939\Downloads\archive (2)\leapGestRecog\01\02_l
Directory 

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d_3 (Conv2D)                    │ (None, 30, 30, 32)          │             320 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_3 (MaxPooling2D)       │ (None, 15, 15, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_4 (Conv2D)                    │ (None, 13, 13, 64)          │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_4 (MaxPooling2D)       │ (None, 6, 6, 64)            │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_5 (Conv2D)                    │ (None, 4, 4, 128)           │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_5 (MaxPooling2D)       │ (None, 2, 2, 128)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_1 (Flatten)                  │ (None, 512)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 128)                 │          65,664 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 10)                  │           1,290 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 159,626 (623.54 KB)

 Trainable params: 159,626 (623.54 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/30
500/500 ━━━━━━━━━━━━━━━━━━━━ 7s 10ms/step - accuracy: 0.5934 - loss: 1.1622 - val_accuracy: 0.9933 - val_loss: 0.0218
Epoch 2/30
500/500 ━━━━━━━━━━━━━━━━━━━━ 4s 8ms/step - accuracy: 0.9846 - loss: 0.0564 - val_accuracy: 0.9980 - val_loss: 0.0063
Epoch 3/30
500/500 ━━━━━━━━━━━━━━━━━━━━ 4s 8ms/step - accuracy: 0.9942 - loss: 0.0225 - val_accuracy: 0.9987 - val_loss: 0.0047
Epoch 4/30
500/500 ━━━━━━━━━━━━━━━━━━━━ 4s 9ms/step - accuracy: 0.9956 - loss: 0.0161 - val_accuracy: 0.9992 - val_loss: 0.0021
Epoch 5/30
500/500 ━━━━━━━━━━━━━━━━━━━━ 4s 8ms/step - accuracy: 0.9913 - loss: 0.0335 - val_accuracy: 1.0000 - val_loss: 6.1039e-04
Epoch 6/30
500/500 ━━━━━━━━━━━━━━━━━━━━ 4s 9ms/step - accuracy: 0.9988 - loss: 0.0061 - val_accuracy: 0.9987 - val_loss: 0.0051
Epoch 7/30
500/500 ━━━━━━━━━━━━━━━━━━━━ 5s 9ms/step - accuracy: 0.9964 - loss: 0.0113 - val_accuracy: 0.9992 - val_loss: 0.0020
Epoch 8/30
500/500 ━━━━━━━━━━━━━━━━━━━━ 4s 9ms/step - accuracy: 0.9981 - loss: 0.0066 - val_accurac